<a href="https://colab.research.google.com/github/MikMik1011/petnica_csgo_ai_project/blob/master/PROJEKAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from sklearn.model_selection import train_test_split
from datetime import datetime

!pip install python-telegram-bot

import telegram
telegrambot = telegram.Bot("5458378014:AAHbuC2qcGiUEqD02i6uWVtuE8y0xP_CzEg")


In [ ]:
def norm(d):
    arr = d.copy()
    for i in range(d.shape[1]): 
      for j in range(d.shape[0]):
        arr[j][i] = d[j][i] - d[0][i]
    return arr

def minmax(d):
    arr = d.copy()
    for f in range(d.shape[2]):
      xmax = d[:,:,f].max()
      xmin = d[:,:,f].min() 
      for i in range(d.shape[0]):
        for j in range(d.shape[1]):
          x = arr[i][j][f]
          arr[i][j][f] = (x - xmin) / (xmax - xmin)
    return arr

In [ ]:
X = np.load('/content/drive/MyDrive/PROJEKAT DATASET/normX.npy')
Y = np.load('/content/drive/MyDrive/PROJEKAT DATASET/Y.npy')

In [ ]:
X = minmax(X)

In [ ]:
np.save('/content/drive/MyDrive/PROJEKAT DATASET/normX.npy', X)

In [ ]:
legitX, cheatX = np.split(X, 2)
legitY, cheatY = np.split(Y, 2)

In [ ]:
ltX, leX, ltY, leY = train_test_split(legitX, legitY, test_size=0.20)
ctX, ceX, ctY, ceY = train_test_split(cheatX, cheatY, test_size=0.20)

In [ ]:
tX = np.concatenate((ltX, ctX))
eX = np.concatenate((leX, ceX))
tY = np.concatenate((ltY, ctY))
eY = np.concatenate((leY, ceY))

In [ ]:
from sklearn.dummy import DummyClassifier

zeror = DummyClassifier()

zeror.fit(tX, tY)
zeror.score(eX, eY)

In [ ]:
def oner(normalize = False):

    accs = []
  for br in range(tX.shape[2]):
    arX = tX[:,:,br]
    arY = np.repeat(tY, arX.shape[1])
    if (normalize):
      arX = norm(arX)
    arX = np.reshape(arX, arX.shape[0] * arX.shape[1])
    unique, unique_counts = np.unique(arX, return_counts = True)

    ar1 = {}
    ar0 = {}

    for i in range(len(arX)):
      if (arY[i] == 1):
        try:
          ar1[arX[i]] += 1
        except:
          ar1[arX[i]] = 1
      else:
          try:
            ar0[arX[i]] += 1
          except:
            ar0[arX[i]] = 1

    freq = []

    for i in unique:
      if (not ar1.get(i)):
        freq.append(ar0[i])
      elif (not ar0.get(i)):
        freq.append(ar1[i])
      else:
        freq.append(max(ar1[i], ar0[i]))

    accs.append(sum(freq) / len(arX))

  return accs

print(oner())
print(oner(normalize = True))

In [ ]:
from tensorflow.keras import backend as K

def custom_f1(y_true, y_pred):    
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        
        recall = TP / (Positives+K.epsilon())    
        return recall 
    
    
    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
        precision = TP / (Pred_Positives+K.epsilon())
        return precision 
    
    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
class CustomCallback(keras.callbacks.Callback):
    
    def on_train_begin(self, logs=None):
        telegrambot.send_message(text=f"Fit started", chat_id=1163033817)
      
    def on_train_end(self, logs=None):
        telegrambot.send_message(text=f"Fit ended", chat_id=1163033817)
    
    def on_epoch_end(self, epoch, logs=None):
      epoch += 1
      telegrambot.send_message(text=f"Epoch {epoch} ended \nValidation accuracy: {logs['val_binary_accuracy']} \nValidation F1-score: {logs['val_custom_f1']} \nValidation loss: {logs['val_loss']}", chat_id=1163033817)


In [ ]:
model = Sequential()
model.add(Input(shape=(192, 5)))
model.add(LSTM(200))
model.add(Dense(1, activation = "sigmoid"))

model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss=keras.losses.BinaryFocalCrossentropy(),
              metrics=[keras.metrics.BinaryAccuracy(),
                       custom_f1,
                       keras.metrics.FalseNegatives(),
                       keras.metrics.FalsePositives()
                       ])


In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/MODELS/LSTM_2022-08-15_19:56', custom_objects={"custom_f1":custom_f1})

In [ ]:
ntX = norm(tX)
neX = norm(eX)

In [ ]:
rtY = tY.reshape((-1,1))
reY = eY.reshape((-1, 1))

In [ ]:
model.summary()
model.fit(tX, tY, batch_size = 25, epochs = 30, validation_data = (eX, eY), shuffle = True, use_multiprocessing = True, verbose = 1, callbacks=[CustomCallback()])
model.save("/content/drive/MyDrive/MODELS/LSTM_" + datetime.now().strftime("%Y-%m-%d_%H:%M"))
telegrambot.send_message(text=f"Model saved", chat_id=1163033817)

In [ ]:
from sklearn.metrics import confusion_matrix#

predY = model.predict(eX)

confMatr = confusion_matrix(eY, np.round(predY, decimals = 0) , normalize='pred')
print(confMatr)

In [ ]:
min(eX[:,:,3].reshape(-1))